In [ ]:
# ───────────────────── 1. LIBRERÍAS ─────────────────────────────
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt, matplotlib.dates as mdates
import matplotlib.colors as mcolors, matplotlib.cm as cm
from matplotlib.collections import LineCollection
from matplotlib.patches import Circle, FancyArrowPatch
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import cartopy.crs as ccrs
from matplotlib import colormaps
from pathlib import Path
import matplotlib.ticker as mticker
from utils import decimal_to_datetime, norm_dep, bezier
from matplotlib.lines import Line2D
import numpy as np
import matplotlib.lines as mlines
from matplotlib.patches import FancyArrowPatch
import random
import matplotlib.cm as cm
import matplotlib.colors as colors
from shapely.geometry import Point
from adjustText import adjust_text
import matplotlib
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib import rcParams

In [ ]:
# ───────────────────── 2. CONFIGURACIÓN DE FUENTES ─────────────────────────────

# Ruta al archivo .ttf de Helvética
helvetica_fp = fm.FontProperties(fname='../font/Helvetica.ttf')
helvetica_name = helvetica_fp.get_name()

print("Nombre de la fuente registrada desde el archivo:", helvetica_name)

In [ ]:
# ───────────────────── 3. CARGA DE DATOS ───────────────────────
# Lee el archivo
file = Path("../data/Fig3_AllPeru_RegionDateMovements.txt")
df   = pd.read_csv(file, sep="\t")

# Convierte fechas decimales a datetime
for col in ("ParentDate", "ChildDate"):
    df[col] = df[col].apply(decimal_to_datetime)

# Normaliza nombres de regiones
for col in ("ParentRegion", "ChildRegion"):
    df[col] = df[col].apply(norm_dep)

# Agrega columnas de tiempo
df['año'] = df['ParentDate'].dt.year
df['mes'] = df['ParentDate'].dt.month
df['trimestre'] = df['ParentDate'].dt.quarter
df = df.sort_values('ParentDate')

# Agrupa movimientos por ruta única
df_count = df.groupby(["ParentRegion", "ChildRegion"]).size().reset_index(name="count")

# Define la normalización para color
min_count = df_count['count'].min()
max_count = df_count['count'].max()
print(f"Min count: {min_count}, Max count: {max_count}")  # ← para verificar que es 1 a 1556

## Normalización logarítmica para el colormap
norm = colors.Normalize(vmin=min_count, vmax=max_count)
# cmap = cm.get_cmap('Purples')
cmap = matplotlib.colormaps['Purples']

In [ ]:
# ───────────────────── 4. SHAPEFILE Y CENTROIDES ───────────────
shp = "../data/DEPARTAMENTOS_inei_geogpsperu_suyopomalia/DEPARTAMENTOS_inei_geogpsperu_suyopomalia.shp"
gdf = gpd.read_file(shp)
# 1.  Proyección métrica adecuada para Perú  ──────────
#    UTM zona 18 Sur (EPSG:32718) cubre la mayor parte del país.
gdf_m = gdf.to_crs("EPSG:32718")
# 2.  Centroides en metros  ───────────────────────────
centroids_xy = gdf_m.centroid            # punto (x, y) en metros
# 3.  De vuelta a lon/lat  ───────────────────────────
centroids_ll = centroids_xy.to_crs("EPSG:4326")   # WGS-84

gdf["NOMBDEP"] = gdf["NOMBDEP"].apply(norm_dep)

# 4.  Diccionario {DEP: (lon, lat)}  ─────────────────
coords = {dep: (pt.x, pt.y)
          for dep, pt in zip(gdf["NOMBDEP"], centroids_ll)}

In [ ]:
# ───────────────────── 5. ESTADÍSTICAS PARA TAMAÑOS Y COLORES ──
donor_total = df["ParentRegion"].value_counts()
max_don     = donor_total.max()
size_lut    = ((donor_total / max_don) ** 0.5) * 900    # escala subjetiva

num_dates   = mdates.date2num(df["ParentDate"])
norm        = mcolors.Normalize(vmin=num_dates.min(), vmax=num_dates.max())
sm          = cm.ScalarMappable(norm=norm, cmap="viridis")

# paleta pastel para relleno de departamentos
N = len(gdf)                     # nº de departamentos
pastel = colormaps.get_cmap("Pastel2").resampled(N)
fill_color = {dep: pastel(i) for i, dep in enumerate(sorted(gdf["NOMBDEP"]))}

In [ ]:
# ───────────────────── 6. FIGURA Y MAPA BASE ───────────────────
fig = plt.figure(figsize=(8, 10), dpi=800)
ax  = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([-84, -68.2, -20.2, 0.2])

# relleno pastel + borde doble (halo blanco + línea negra)
for dep, geom in zip(gdf["NOMBDEP"], gdf.geometry):
    ax.add_geometries([geom], ccrs.PlateCarree(),
                      # facecolor=fill_color[dep],
                      facecolor="gainsboro",     # <-- siempre el mismo
                      edgecolor="white",
                      linewidth=2.4, zorder=0)
    ax.add_geometries([geom], ccrs.PlateCarree(),
                      facecolor="none",
                      edgecolor="black",
                      linewidth=0.8, zorder=1)

In [ ]:
# ───────────────────── 7. ARCOS Y FLECHAS ──────────────────────
# Asegúrate de que la normalización logarítmica esté definida aquí
min_count = df_count['count'].min()
max_count = df_count['count'].max()
norm_log = colors.LogNorm(vmin=min_count, vmax=max_count)
# Cambia el colormap a uno que empiece en blanco y termine fuerte, por ejemplo 'Reds'
# cmap = cm.get_cmap('Purples') # O 'Blues', 'Purples', 'Oranges', etc.
cmap = matplotlib.colormaps['Purples']

# Ordena df_count por 'count' en orden descendente
df_count_sorted = df_count.sort_values(by='count', ascending=False)

# Define un zorder máximo para las flechas
max_arrow_zorder = 100 # Ajusta este valor si es necesario

def bezier(x_start, y_start, x_end, y_end, rad=0.2, n_points=100):
    """
    Genera puntos de una curva de Bézier cuadrática entre dos puntos,
    con una curvatura especificada por `rad` y una cantidad de puntos `n_points`.
    """
    # Punto de control para la curva (curvatura)
    ctrl_x = (x_start + x_end) / 2 - rad * (y_end - y_start)
    ctrl_y = (y_start + y_end) / 2 + rad * (x_end - x_start)

    t = np.linspace(0, 1, n_points)
    x_vals = (1 - t)**2 * x_start + 2 * (1 - t) * t * ctrl_x + t**2 * x_end
    y_vals = (1 - t)**2 * y_start + 2 * (1 - t) * t * ctrl_y + t**2 * y_end

    return x_vals, y_vals

for _, row in df_count_sorted.iterrows():
    parent = row["ParentRegion"]
    child = row["ChildRegion"]
    count = row["count"]

    x_start, y_start = coords[parent]
    x_end, y_end = coords[child]

    rad = random.choice([-0.3, -0.2, -0.1, 0.1, 0.2, 0.3])
    color = cmap(norm_log(count))

    alpha_value = norm_log(count)
    alpha_value = max(0.1, min(0.9, alpha_value))

    if count > 1000:
        linewidth_value = 1.5
    elif count >= 500:
        linewidth_value = 1.0
    else:
        linewidth_value = 0.5

    zorder_value = max_arrow_zorder * (count - min_count) / (max_count - min_count)
    zorder_value = max(1, zorder_value)

    # (1) Dibuja curva con función bezier:
    x_vals, y_vals = bezier(x_start, y_start, x_end, y_end, rad, n_points=100)
    ax.plot(x_vals, y_vals, color=color, linewidth=linewidth_value, alpha=alpha_value,
            transform=ccrs.PlateCarree(), zorder=zorder_value)

    # (2) Flecha en punto medio de la curva:
    mid_idx = len(x_vals) // 2
    ax.annotate("",
        xy=(x_vals[mid_idx + 1], y_vals[mid_idx + 1]),
        xytext=(x_vals[mid_idx - 1], y_vals[mid_idx - 1]),
        arrowprops=dict(arrowstyle="->", color=color, lw=linewidth_value),
        annotation_clip=False,
        xycoords=ccrs.PlateCarree()._as_mpl_transform(ax),
        textcoords=ccrs.PlateCarree()._as_mpl_transform(ax),
        zorder=zorder_value + 1
    )

In [ ]:
# ───────────────────── 8. PUNTOS DE DENSIDAD DETALLADA ───────
try:
    df_densidad = pd.read_csv("../data/densidad_poblacional.csv")

    if 'Departamento' in df_densidad.columns:
        df_densidad['Departamento'] = df_densidad['Departamento'].apply(norm_dep)
    else:
        print("Advertencia: Columna 'Departamento' no encontrada en el archivo de densidad detallada.")

    # Crea un diccionario {Departamento: Densidad}
    densidad_lut = df_densidad.set_index('Departamento')['Densidad'].to_dict()

    # Define cuántas personas representa cada punto
    personas_por_punto = 10000

    # Lista para almacenar las coordenadas de todos los puntos a dibujar
    all_density_points = []

    # Define el color y tamaño de los puntos de densidad
    punto_color = 'grey'
    punto_size = 0.5

    # Define un zorder para estos puntos de densidad
    density_points_zorder = 3


    # Itera sobre la geometría de cada departamento
    for dep, geom in zip(gdf["NOMBDEP"], gdf.geometry):
        if dep in densidad_lut:
            densidad_valor = densidad_lut[dep]

            # Calcula el número de puntos para este departamento
            num_points = int(densidad_valor / personas_por_punto)

            # Genera puntos aleatorios dentro de la geometría del departamento
            minx, miny, maxx, maxy = geom.bounds # Límites del departamento

            generated_points = []
            while len(generated_points) < num_points:
                # Genera una coordenada aleatoria dentro de los límites
                random_x = random.uniform(minx, maxx)
                random_y = random.uniform(miny, maxy)
                point = Point(random_x, random_y)

                # Verifica si el punto está dentro de la geometría del departamento
                if geom.contains(point):
                    generated_points.append((random_x, random_y))

            all_density_points.extend(generated_points)

    # Dibuja todos los puntos de densidad
    if all_density_points:
        x_coords, y_coords = zip(*all_density_points)
        ax.scatter(x_coords, y_coords, color=punto_color, s=punto_size, zorder=density_points_zorder, transform=ccrs.PlateCarree())
        print(f"Dibujados {len(all_density_points)} puntos de densidad.")
    else:
        print("No se generaron puntos de densidad.")

except FileNotFoundError:
    print("Error: El archivo 'densidad_poblacional.csv' no fue encontrado.")
except KeyError as e:
    print(f"Error de columna en el archivo de densidad: {e}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

In [ ]:
# ───────────────────── 9. PUNTOS Y ETIQUETAS ───────────────────
FACE = (0.886, 0.290, 0.200, 0.8)  # naranja-rojo 80 %
label_zorder = max_arrow_zorder + 10 # Un valor mayor que el zorder máximo de las flechas

# Lista para almacenar los objetos de texto
texts = []

for dep, (lon, lat) in coords.items():
    size_px = size_lut.get(dep, 80)
    circ = Circle((lon, lat), size_px/2200, facecolor=FACE,
                  edgecolor="white", lw=0.6, zorder=4,
                  transform=ccrs.PlateCarree())
    ax.add_patch(circ)

    # --- Ajuste Manual de Coordenadas para Departamentos Específicos ---
    adjusted_lon = lon
    adjusted_lat = lat
    offset_lat = 0.15 if lat > -10 else -0.15 # Offset vertical base
    offset_lon = 0.05 # Offset horizontal base

    # Ajustes específicos basados en la observación de los solapamientos
    if dep == 'Lambayeque':
        adjusted_lon -= 0.5 # Mover ligeramente a la izquierda
        adjusted_lat += 0.5 # Mover ligeramente hacia arriba
    elif dep == 'Cajamarca':
        adjusted_lon += 0.35 # Mover ligeramente a la derecha
        adjusted_lat -= 0.35 # Mover ligeramente hacia abajo
    elif dep == 'Lima':
         adjusted_lon += 0.15 # Mover más a la derecha
         offset_lat += 0.1 # Añadir un offset vertical extra
    elif dep == 'Callao':
         adjusted_lon -= 0.15 # Mover más a la izquierda
         offset_lat -= 0.1 # Añadir un offset vertical extra (negativo)
    elif dep == 'Ayacucho':
         adjusted_lon -= 0.1 # Mover ligeramente a la izquierda
         offset_lat += 0.08 # Añadir un offset vertical extra
    elif dep == 'Apurimac':
         adjusted_lon += 0.1 # Mover ligeramente a la derecha
         offset_lat -= 0.08 # Añadir un offset vertical extra (negativo)

    # Aplica el offset (base + manual si aplica)
    final_lon = adjusted_lon + offset_lon
    final_lat = adjusted_lat + offset_lat

    # Crea el objeto de texto con las coordenadas ajustadas
    texts.append(ax.text(final_lon, final_lat, dep.title(),
                          fontsize=8, ha="left", va="center", zorder=label_zorder,
                          bbox=dict(facecolor="white", alpha=.7, pad=1, lw=0),
                          transform=ccrs.PlateCarree(),
                          fontproperties=helvetica_fp))

# Después del bucle, llama a adjust_text con parámetros de force_points y force_text razonables
adjust_text(texts, ax=ax,
            force_points=(1.0, 1.0), # Ajusta estos valores según necesites la separación general
            force_text=(1.0, 1.0),
            expand_points=(1.1, 1.1),
            expand_text=(1.1, 1.1)
            # arrowprops=dict(arrowstyle='-', color='none')
            ) # Opciones para el ajuste

In [ ]:
# ───────────────────── 10. ESCALA 200 km ───────────────────────
bar = AnchoredSizeBar(ax.transData, 1.2, '200 km', loc=4,
                      pad=0.6, color='black',
                      frameon=False, size_vertical=0.10,
                      fontproperties=helvetica_fp)
ax.add_artist(bar)

In [ ]:
# ───────────────────── 11. TÍTULO Y PIE ────────────────────────
plt.suptitle("Dispersión de linajes virales entre departamentos del Perú (2020-2024)",
             fontsize=13, fontweight='bold', y=0.988,
             fontproperties=helvetica_fp)
bbox = ax.get_position()
# caption = (
#     "Figura 1 | Arcos coloreados según la fecha origen del evento (ParentDate). Los círculos se escalan al número total de eventos registrados \n"
#     "como origen para cada departamento. El fondo muestra los límites administrativos proporcionados por el INEI."
# )
# Bbox(x0, y0, x1, y1)
# Opcional: envolver a ~90 caracteres para evitar líneas ultra-largas
# wrapped = "\n".join(textwrap.wrap(caption, 90))
# ── 3. Coordenadas del centro del mapa ─────────────────────────
# x_center = bbox.x0 + bbox.width / 2
# y_bottom = 0.01                # distancia vertical en fracción de figura
# ── 4. Dibujar texto centrado y con alineación adecuada ────────
# fig.text(
#     x_center, y_bottom,
#     caption,
#     ha="center", va="bottom",
#     fontsize=6.5,
#     wrap=True                  # Matplotlib ≥3.4 respeta saltos automáticos
# )

In [ ]:
# ───────────────────── 12. LEYENDA COMBINADA ────────────────────────

# Define los rangos de cantidad y sus propiedades de visualización para las flechas
legend_ranges_arrows = [
    (1, 50, 0.5, 'Low'),
    (51, 500, 0.8, 'Medium-Low'),
    (501, 1000, 1.2, 'Medium-High'),
    (1001, max_count, 1.8, 'High')
]

# Crea una lista de manejadores de leyenda personalizados para las flechas
legend_handles_arrows = []
legend_labels_arrows = []

# Asegúrate de tener definida la normalización y el colormap
min_count = df_count['count'].min()
max_count = df_count['count'].max()
norm_log = colors.LogNorm(vmin=min_count, vmax=max_count)
# cmap = cm.get_cmap('Purples')
cmap = matplotlib.colormaps['Purples']

for min_val, max_val, linewidth, label_range in legend_ranges_arrows:
    representative_count = (min_val + max_val) / 2
    try:
        color = cmap(norm_log(representative_count))
    except ValueError:
        if representative_count < min_count:
            color = cmap(norm_log(min_count))
        else:
            color = cmap(norm_log(max_count))

    handle = Line2D([0], [0], color=color, lw=linewidth)
    legend_handles_arrows.append(handle)
    legend_labels_arrows.append(f'{label_range}: {min_val}-{max_val}')

# --- Leyenda para Puntos de Densidad Poblacional ---
legend_point_color = 'grey'
legend_point_size = 10
legend_density_handle = Line2D([0], [0], marker='o', color='none', label='10,000 people',
                       markerfacecolor=legend_point_color, markeredgecolor='none', markeredgewidth= 0, markersize=legend_point_size, linestyle='none')

# --- Leyenda para Círculos Rojos (Origen de Dispersión) ---

# Define tamaños representativos para la leyenda de los círculos
representative_sizes = [
    size_lut.min(), # Tamaño mínimo
    size_lut.mean(), # Tamaño medio (aproximado)
    size_lut.max()  # Tamaño máximo
]

# Convertir los tamaños representativos de px (size_lut) a un tamaño para la leyenda (markersize)
legend_size_factor = 0.02

legend_handles_circles = []
legend_labels_circles = []

# Usa un color representativo para los círculos (FACE)
circle_color = FACE

for size in representative_sizes:
    # Convierte el tamaño de size_lut a markersize para la leyenda
    legend_markersize = size * legend_size_factor

    handle = Line2D([0], [0], marker='o', color='none',
                       markerfacecolor=circle_color, markeredgecolor='white', markeredgewidth=0.6, markersize=legend_markersize, linestyle='None')
    legend_handles_circles.append(handle)
    # Agrega etiquetas descriptivas para cada tamaño
    if size == size_lut.min():
        label = 'Low Dispersion'
    elif size == size_lut.max():
        label = 'High Dispersion'
    else:
        label = 'Medium Dispersion'
    legend_labels_circles.append(label)


# --- Creación de las Leyendas ---

# Agrega la leyenda de densidad poblacional
legend_density = ax.legend(handles=[legend_density_handle], title="Population Density",
                           loc='lower left',
                           bbox_to_anchor=(0, 0.215),   # Ajusta la posición
                           prop=helvetica_fp)

ax.add_artist(legend_density)

# Agrega la leyenda para los círculos rojos (Origen de Dispersión) con tamaño variable
legend_circles = ax.legend(handles=legend_handles_circles, labels=legend_labels_circles,
                           title="Department Origin Size",
                           loc='lower left',
                           bbox_to_anchor=(0, 0.095),
                           prop=helvetica_fp)

ax.add_artist(legend_circles)

# Crea la leyenda combinada (color y grosor de las flechas)
legend_combined = ax.legend(handles=legend_handles_arrows, labels=legend_labels_arrows,
                            title="Number of Dispersions (Route)",
                            loc='lower left',
                            bbox_to_anchor=(0, 0.00), # Ajusta la posición
                            ncol=2,
                            frameon=True,
                            prop=helvetica_fp)

ax.add_artist(legend_combined)

In [ ]:
# ───────────────────── 13. GUARDAR Y MOSTRAR ───────────────────
fig.savefig("../output/Movimientos_Departamentales_Peru_año.png", dpi=800, bbox_inches="tight")
plt.show()
fig

In [ ]:
# ───────────────────── ADICIONAL #1. CONTEO DE MOVIMIENTOS POR RUTA ───────────────────

# Carga del archivo
df = pd.read_csv("../data/Fig3_AllPeru_RegionDateMovements.txt", sep="\t")  # Usa sep="," si fuera CSV

# Cuenta el número de movimientos por ruta (origen-destino)
# Usar los nombres correctos de las columnas: 'ParentRegion' y 'ChildRegion'
ruta_counts = df.groupby(['ParentRegion', 'ChildRegion']).size().reset_index(name='cantidad_de_movimientos')

# Ordena por cantidad
ruta_counts = ruta_counts.sort_values(by='cantidad_de_movimientos', ascending=False)

print(ruta_counts)

In [ ]:
# ───────────────────── ADICIONAL #2. HISTOGRAMA DE MOVIMIENTOS POR RUTA ───────────────────

import matplotlib.pyplot as plt
import seaborn as sns # Opcional, para gráficos más bonitos

plt.figure(figsize=(8, 4))
# sns.histplot(df_count['count'], bins=50, kde=True) # Con seaborn
plt.hist(df_count['count'], bins=50) # Con matplotlib básico
plt.title('Distribución del número de movimientos por ruta')
plt.xlabel('Número de movimientos')
plt.ylabel('Frecuencia')
plt.yscale('log') # Usa escala logarítmica en y si hay muchos valores bajos
plt.grid(True, alpha=0.5)
plt.show()